## COD API, One Match --> explore API, format, clean & reshape

Activision Call of Duty API use case for **Match** endpoint, using a slightly amended version of callofduty.py client and custom cleaning functions (from wzkd app)

In [3]:
import asyncio
from pathlib import Path
import os
import sys
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle

import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType, Language

#### Login using SSO

In [2]:
# We're storing our SSO token in an .env file stored locally to separate our conf from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

#### Slightly modify client method to call a given match

In [3]:
# 1. import client classes
import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request

# 2. define a modified method
async def GetMatchStats(
    self, platform, title: Title, mode: Mode, matchId: int, language: Language = Language.English, **kwargs
):
    """ 
    Compared to client : modified so that we do not use Platform.abc as parameter
    but instead our app-defined workflow (drop down menu) to select our platform of choice"
    """
    return (
        await self.http.GetFullMatch(
            title.value, platform, mode.value, matchId, language.value
        )
    )["data"]["allPlayers"]
    # api result, at very least for Warzone {'data':{'all_players:' is the only key},'status': call status}

# 3. add modified method in callofduty.py client at runtime
Client.GetMatchStats = GetMatchStats

# if you wanted to retrieve some ids, here the function as well :
async def GetMatches(self, platform, username: str, title: Title, mode: Mode, **kwargs):
    """
    Returns matches history, notably matches Ids
    """

    limit: int = kwargs.get("limit", 20)
    startTimestamp: int = kwargs.get("startTimestamp", 0)
    endTimestamp: int = kwargs.get("endTimestamp", 0)

    data: dict = (
        await self.http.GetPlayerMatches(
            platform,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp,
        )
    )[
        "data"
    ]  # originally (in callofduty.py client), API result was truncated here

    return data
Client.GetMatches = GetMatches

#### Option : get Matches Ids

In [4]:
ids = await client.GetMatches('battle', 'amadevs#1689', Title.ModernWarfare, Mode.Warzone)
df_ids = pd.DataFrame(ids)
df_ids.head(4)

,platform,title,timestamp,type,matchId,map
0,battle,mw,1653146158000,6552125305277136,3213166026030624400,7688482426490492
1,battle,mw,1653145222000,6552125305277136,15216902894198084690,7688482426490492
2,battle,mw,1653143689000,6552125305277136,2061409765938289321,7688482426490492
3,battle,mw,1653142581000,6552125305277136,4101952639478595012,7688482426490492


#### Get Match data

In [5]:
match = await client.GetMatchStats('battle', Title.ModernWarfare, Mode.Warzone, matchId=1653145222000)

# example of IDs :
# 10717821121770145230 : BR duos, Season 5 april 2022
# 1245830289127567228 : BR trios, with me 6 kills
# 5010761031247628577 : BR trios 
# 11672696746036290501 a "custom" mode game (Rumble Clash)
# 6825832239054239925 : Iron Trials Trios

##### Option: save previous result so we're not getting annoyed by API rate limits or inconsistencies -,-

## Match result : structure

In [6]:
# load previously saved data
with open('data/match_br_1.pkl', 'rb') as f:
    match = pickle.load(f)

### Overview : dict --> df

In [7]:
df_match = pd.DataFrame(match)
display(df_match.head(2))
keys = list(df_match.keys())
keys.sort()
pprint(keys)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1652911897,1652913521,mp_wz_island,br_brduos,17873325042825069645,1624000,None,1,wz,144,"{'kills': 1.0, 'medalXp': 10.0, 'matchXp': 292...","{'team': 'team_thirteen', 'rank': 54.0, 'award...",70,None,False,False
1,1652911897,1652913521,mp_wz_island,br_brduos,17873325042825069645,1624000,None,1,wz,144,"{'kills': 0.0, 'medalXp': 20.0, 'matchXp': 494...","{'team': 'team_nine', 'rank': 37.0, 'awards': ...",70,None,False,False


['draw',
 'duration',
 'gameType',
 'map',
 'matchID',
 'mode',
 'player',
 'playerCount',
 'playerStats',
 'playlistName',
 'privateMatch',
 'rankedTeams',
 'teamCount',
 'utcEndSeconds',
 'utcStartSeconds',
 'version']


### API structure for one player (a 'row')

##### Reminder : match endpoint returns n players stats (typically 150+ in Battle Royale) as rows, for a single --queried match

In [8]:
pprint(match[0:1], depth=3)

[{'draw': False,
  'duration': 1624000,
  'gameType': 'wz',
  'map': 'mp_wz_island',
  'matchID': '17873325042825069645',
  'mode': 'br_brduos',
  'player': {'awards': {...},
             'brMissionStats': {...},
             'clantag': 'JP',
             'loadout': [...],
             'loadouts': [...],
             'rank': 54.0,
             'team': 'team_thirteen',
             'uno': '15695009743495759618',
             'username': 'JanPancir'},
  'playerCount': 144,
  'playerStats': {'assists': 0.0,
                  'bonusXp': 0.0,
                  'challengeXp': 0.0,
                  'damageDone': 583.0,
                  'damageTaken': 510.0,
                  'deaths': 2.0,
                  'distanceTraveled': 312685.78,
                  'executions': 0.0,
                  'gulagDeaths': 0.0,
                  'gulagKills': 1.0,
                  'headshots': 0.0,
                  'kdRatio': 0.5,
                  'kills': 1.0,
                  'longestStreak': 2.0,
   

### Focus : what's in 'playerStats' ?

In [9]:
player_stats = df_match['playerStats'].apply(pd.Series)
display(player_stats.head(5))
pprint(player_stats.keys())

,kills,medalXp,matchXp,scoreXp,wallBangs,score,totalXp,headshots,assists,challengeXp,...,longestStreak,teamPlacement,damageDone,damageTaken,objectiveLastStandKill,objectiveBrDownEnemyCircle1,objectiveBrMissionPickupTablet,objectiveReviver,objectiveBrKioskBuy,objectiveBrCacheOpen
0,1.0,10.0,2920.0,825.0,0.0,825.0,4290.0,0.0,0.0,0.0,...,2.0,53.0,583.0,510.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,20.0,4947.0,975.0,0.0,800.0,6137.0,0.0,0.0,0.0,...,0.0,37.0,66.0,517.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,28.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,10.0,1772.0,525.0,0.0,425.0,2853.0,0.0,0.0,0.0,...,1.0,62.0,564.0,300.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,368.0,0.0,0.0,0.0,368.0,0.0,0.0,0.0,...,0.0,69.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Index(['kills', 'medalXp', 'matchXp', 'scoreXp', 'wallBangs', 'score',
       'totalXp', 'headshots', 'assists', 'challengeXp', 'rank',
       'scorePerMinute', 'distanceTraveled', 'teamSurvivalTime', 'deaths',
       'kdRatio', 'bonusXp', 'gulagDeaths', 'timePlayed', 'executions',
       'gulagKills', 'nearmisses', 'percentTimeMoving', 'miscXp',
       'longestStreak', 'teamPlacement', 'damageDone', 'damageTaken',
       'objectiveLastStandKill', 'objectiveBrDownEnemyCircle1',
       'objectiveBrMissionPickupTablet', 'objectiveReviver',
       'objectiveBrKioskBuy', 'objectiveBrCacheOpen'],
      dtype='object')


In [10]:
# some cols may not appear re. the mode you're playing in
cols_set1 = [
    'kills',
    'medalXp',
    'score',
    'matchXp',
    'scoreXp',
    'scorePerMinute',
    'challengeXp',
    'totalXp'
]
cols_set2 = [
    'kills',
     'deaths',
     'assists',
     'kdRatio',
     'headshots',
     'percentTimeMoving',
     'nearmisses',
     'executions',
     'longestStreak',
     'damageDone',
     'damageTaken'
]
cols_set3 = [
    'kills',
    'timePlayed',
    'distanceTraveled',
    'teamSurvivalTime',
    'teamPlacement'
]
cols_set4 = [
    'kills',
    'objectiveTeamWiped',
    'objectiveLastStandKill',
    'objectiveBrDownEnemyCircle1',
    'objectiveBrDownEnemyCircle2',
    'objectiveBrDownEnemyCircle3',
    'objectiveBrDownEnemyCircle4',
    'objectiveBrDownEnemyCircle5',
    'objectiveBrDownEnemyCircle6',
]
cols_set5 = [
    'kills',
    'objectiveReviver',
    'objectiveBrMissionPickupTablet',
    'objectiveBrCacheOpen',
    'objectiveBrKioskBuy'
]
cols_set5  = [
    'kills',
    'objectiveMunitionsBoxTeammateUsed',
    'objectiveMedalScoreSsKillTomaStrike',
    'objectiveDestroyedVehicleHeavy',
    'objectiveDestroyedVehicleMedium'
]

In [11]:
for set_cols in [cols_set1, cols_set2, cols_set3, cols_set4, cols_set5]:
    # some colums (keys) not existing depending on gamemode /  or all values missing
    set_cols = [col for col in set_cols if col in player_stats.columns.tolist()] 
    display(player_stats[set_cols].sort_values('kills', ascending=False).head(10))

,kills,medalXp,score,matchXp,scoreXp,scorePerMinute,challengeXp,totalXp
68,17.0,880.0,16400.0,12778.0,17970.0,570.765661,0.0,41115.0
15,11.0,570.0,11225.0,12778.0,12570.0,392.253931,0.0,32811.0
55,9.0,345.0,6900.0,11925.0,7775.0,274.717983,0.0,24737.0
57,8.0,350.0,7550.0,12062.0,7750.0,296.272073,0.0,27183.0
97,8.0,350.0,5900.0,8767.0,6100.0,360.856269,0.0,20001.0
49,7.0,320.0,4350.0,8282.0,4650.0,277.955272,0.0,15642.0
137,7.0,305.0,6850.0,10995.0,7150.0,305.576208,0.0,21359.0
108,7.0,55.0,5375.0,12433.0,5675.0,193.461308,0.0,20894.0
115,7.0,390.0,3775.0,8385.0,4075.0,237.670514,0.0,16175.0
13,7.0,315.0,7275.0,11925.0,8150.0,284.179688,0.0,23667.0


,kills,deaths,assists,kdRatio,headshots,percentTimeMoving,nearmisses,executions,longestStreak,damageDone,damageTaken
68,17.0,2.0,5.0,8.500000,1.0,96.670940,0.0,0.0,14.0,7910.0,957.0
15,11.0,2.0,5.0,5.500000,1.0,96.350840,0.0,0.0,6.0,5215.0,1141.0
55,9.0,3.0,2.0,3.000000,4.0,95.288330,0.0,0.0,5.0,4687.0,1026.0
57,8.0,1.0,2.0,8.000000,6.0,93.079580,0.0,0.0,8.0,4589.0,646.0
97,8.0,2.0,1.0,4.000000,3.0,94.696970,0.0,0.0,6.0,3264.0,705.0
49,7.0,3.0,0.0,2.333333,0.0,66.125290,0.0,0.0,6.0,2980.0,747.0
137,7.0,2.0,1.0,3.500000,3.0,90.981010,0.0,0.0,5.0,2019.0,888.0
108,7.0,3.0,0.0,2.333333,1.0,72.663250,0.0,0.0,3.0,3173.0,1817.0
115,7.0,2.0,0.0,3.500000,5.0,94.965675,0.0,0.0,5.0,3544.0,754.0
13,7.0,2.0,3.0,3.500000,2.0,95.583170,0.0,0.0,7.0,2932.0,964.0


,kills,timePlayed,distanceTraveled,teamSurvivalTime,teamPlacement
68,17.0,1724.0,379477.84,1578912.0,2.0
15,11.0,1717.0,368702.97,1578912.0,2.0
55,9.0,1507.0,391874.38,1434096.0,5.0
57,8.0,1529.0,420563.84,1444656.0,4.0
97,8.0,981.0,245410.56,919824.0,17.0
49,7.0,939.0,402788.25,869280.0,20.0
137,7.0,1345.0,514926.72,1269504.0,8.0
108,7.0,1667.0,382399.78,1515120.0,3.0
115,7.0,953.0,400332.88,871104.0,19.0
13,7.0,1536.0,436141.16,1434096.0,5.0


,kills,objectiveLastStandKill,objectiveBrDownEnemyCircle1
68,17.0,NaN,NaN
15,11.0,NaN,NaN
55,9.0,NaN,NaN
57,8.0,NaN,NaN
97,8.0,NaN,NaN
49,7.0,NaN,NaN
137,7.0,NaN,NaN
108,7.0,NaN,NaN
115,7.0,NaN,NaN
13,7.0,NaN,NaN


,kills
68,17.0
15,11.0
55,9.0
57,8.0
97,8.0
49,7.0
137,7.0
108,7.0
115,7.0
13,7.0


Quick notes :</br>
- objectiveTeamWiped = teams wiped
- objectiveLastStandKill = kills when alone in team ?? ^_o

### Focus : 'player', a nested entry

In [12]:
player = df_match['player'].apply(pd.Series)
display(player.head(5))
pprint(player.keys())

,team,rank,awards,username,uno,clantag,loadouts,brMissionStats,loadout
0,team_thirteen,54.0,"{'mode_x_eliminate': 233712.0, 'save_teammate'...",JanPancir,15695009743495759618,JP,[{'primaryWeapon': {'name': 'iw8_sm_t9handling...,"{'missionsComplete': 0, 'totalMissionXpEarned'...",[{'primaryWeapon': {'name': 'iw8_sm_t9handling...
1,team_nine,37.0,"{'pointblank': 0.0, 'headshot': 0.0}",HardSkillWow,3254616672146851841,NaN,[{'primaryWeapon': {'name': 'iw8_ar_t9standard...,"{'missionsComplete': 0, 'totalMissionXpEarned'...",[{'primaryWeapon': {'name': 'iw8_ar_t9standard...
2,team_sixteen,54.0,{},wzBalou,16494720098789116731,TTV,[],"{'missionsComplete': 0, 'totalMissionXpEarned'...",[]
3,team_sixty_six,54.0,"{'mode_x_eliminate': 89280.0, 'kill_jumper': 0.0}",uysienki,13349743653476921527,ARTE,"[{'primaryWeapon': {'name': 's4_ar_stango44', ...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_ar_stango44', ..."
4,team_sixty_three,54.0,{},User73873564,10943659199620843672,NaN,"[{'primaryWeapon': {'name': 's4_ar_stango44', ...","{'missionsComplete': 0, 'totalMissionXpEarned'...","[{'primaryWeapon': {'name': 's4_ar_stango44', ..."


Index(['team', 'rank', 'awards', 'username', 'uno', 'clantag', 'loadouts',
       'brMissionStats', 'loadout'],
      dtype='object')


#### Inside 'player' entry, 'loadout' is a (list of) list of dict

In [13]:
# Each entry of 'loadout' (or loadouts, they are the same) is a list of dict. Either one dict (if 1 loadout) or more (if you succeed in buying several loadouts)
player_index = 1
pprint(player['loadout'][player_index][0], depth=2)

{'extraPerks': [{...}, {...}, {...}],
 'killstreaks': [{...}, {...}, {...}],
 'lethal': {'image': None,
            'imageLarge': None,
            'label': None,
            'name': 'equip_molotov',
            'progressionImage': None},
 'perks': [{...}, {...}, {...}],
 'primaryWeapon': {'attachments': [...],
                   'imageIcon': None,
                   'imageLoot': None,
                   'label': None,
                   'name': 'iw8_ar_t9standard',
                   'variant': '0'},
 'secondaryWeapon': {'attachments': [...],
                     'imageIcon': None,
                     'imageLoot': None,
                     'label': None,
                     'name': 's4_sm_mpapa40',
                     'variant': '0'},
 'tactical': {'image': None,
              'imageLarge': None,
              'label': None,
              'name': 'equip_hb_sensor',
              'progressionImage': None}}


#### Inside 'player' entry, 'brMissionStats' is a (list of) dict

In [14]:
player['brMissionStats'].apply(pd.Series).head(5)

,missionsComplete,totalMissionXpEarned,totalMissionWeaponXpEarned,missionStatsByType
0,0,0.0,0.0,{}
1,0,0.0,0.0,{}
2,0,0.0,0.0,{}
3,0,0.0,0.0,{}
4,0,0.0,0.0,{}


In [15]:
player['brMissionStats'].apply(pd.Series)[['missionStatsByType']]['missionStatsByType'].apply(pd.Series).head(10)

,domination,scavenger,masterassassination,assassination,timedrun,vip,supply,sabotage
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"{'weaponXp': 650.0, 'xp': 650.0, 'count': 1.0}","{'weaponXp': 500.0, 'xp': 500.0, 'count': 1.0}",NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Format & clean API **match** results using customized tools (wzkd app)

#### Inside 'player' entry, 'loadout' is a (list of) list of dict

In [4]:
# import conf / parse files & methods from wzkd app
import json
import toml

sys.path.insert(0, "..")
from src.utils import load_labels, load_conf
from src.api_format import res_to_df, format_df, augment_df

In [5]:
# conf and labels files stored here as well.
# labels is needed for parsing games modes/weapons, conf stores values such as n of loadouts to extract or columns names
filepath_labels = Path.cwd().parent / "src" / "wz_labels.json"
LABELS = load_labels(filepath_labels)
pprint(LABELS, depth=2)

filepath_conf = Path.cwd().parent / "src" / "conf.toml"
CONF = load_conf(filepath_conf)
pprint(CONF, depth=2)

In [6]:
with open('data/match_br_1.pkl', 'rb') as f:
    res = pickle.load(f)
tmp = pd.DataFrame(res)
display(tmp.head(2))
tmp.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1652911897,1652913521,mp_wz_island,br_brduos,17873325042825069645,1624000,None,1,wz,144,"{'kills': 1.0, 'medalXp': 10.0, 'matchXp': 292...","{'team': 'team_thirteen', 'rank': 54.0, 'award...",70,None,False,False
1,1652911897,1652913521,mp_wz_island,br_brduos,17873325042825069645,1624000,None,1,wz,144,"{'kills': 0.0, 'medalXp': 20.0, 'matchXp': 494...","{'team': 'team_nine', 'rank': 37.0, 'awards': ...",70,None,False,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   utcStartSeconds  144 non-null    int64 
 1   utcEndSeconds    144 non-null    int64 
 2   map              144 non-null    object
 3   mode             144 non-null    object
 4   matchID          144 non-null    object
 5   duration         144 non-null    int64 
 6   playlistName     0 non-null      object
 7   version          144 non-null    int64 
 8   gameType         144 non-null    object
 9   playerCount      144 non-null    int64 
 10  playerStats      144 non-null    object
 11  player           144 non-null    object
 12  teamCount        144 non-null    int64 
 13  rankedTeams      0 non-null      object
 14  draw             144 non-null    bool  
 15  privateMatch     144 non-null    bool  
dtypes: bool(2), int64(6), object(8)
memory usage: 16.2+ KB


In [19]:
# flatten-expand into a DataFrame the result from COD API, for matches history
df_match = res_to_df(res, CONF)
display(df_match.head(3))
df_match.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,totalMissionXpEarned,totalMissionWeaponXpEarned,domination,scavenger,masterassassination,assassination,timedrun,vip,supply,sabotage
0,1652911897,1652913521,mp_wz_island,br_brduos,17873325042825069645,1624000,1,wz,144,70,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1652911897,1652913521,mp_wz_island,br_brduos,17873325042825069645,1624000,1,wz,144,70,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1652911897,1652913521,mp_wz_island,br_brduos,17873325042825069645,1624000,1,wz,144,70,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 65 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   utcStartSeconds                 144 non-null    int64  
 1   utcEndSeconds                   144 non-null    int64  
 2   map                             144 non-null    object 
 3   mode                            144 non-null    object 
 4   matchID                         144 non-null    object 
 5   duration                        144 non-null    int64  
 6   version                         144 non-null    int64  
 7   gameType                        144 non-null    object 
 8   playerCount                     144 non-null    int64  
 9   teamCount                       144 non-null    int64  
 10  draw                            144 non-null    bool   
 11  privateMatch                    144 non-null    bool   
 12  kills                           144 

In [20]:
# make the stats human-readable and parse some values (weapons, games modes)
df_formatted = format_df(df_match,CONF, LABELS)
display(df_formatted)
df_formatted.info()

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,version,gameType,playerCount,teamCount,...,totalMissionXpEarned,totalMissionWeaponXpEarned,domination,scavenger,masterassassination,assassination,timedrun,vip,supply,sabotage
0,2022-05-19 00:11:37,2022-05-19 00:38:41,mp_wz_island,Duos,17873325042825069645,27,1,wz,144,70,...,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
1,2022-05-19 00:11:37,2022-05-19 00:38:41,mp_wz_island,Duos,17873325042825069645,27,1,wz,144,70,...,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
2,2022-05-19 00:11:37,2022-05-19 00:38:41,mp_wz_island,Duos,17873325042825069645,27,1,wz,144,70,...,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
3,2022-05-19 00:11:37,2022-05-19 00:38:41,mp_wz_island,Duos,17873325042825069645,27,1,wz,144,70,...,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
4,2022-05-19 00:11:37,2022-05-19 00:38:41,mp_wz_island,Duos,17873325042825069645,27,1,wz,144,70,...,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2022-05-19 00:11:37,2022-05-19 00:38:41,mp_wz_island,Duos,17873325042825069645,27,1,wz,144,70,...,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
140,2022-05-19 00:11:37,2022-05-19 00:38:41,mp_wz_island,Duos,17873325042825069645,27,1,wz,144,70,...,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN
141,2022-05-19 00:11:37,2022-05-19 00:38:41,mp_wz_island,Duos,17873325042825069645,27,1,wz,144,70,...,1000.0,1000.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,"{'weaponXp': 1000.0, 'xp': 1000.0, 'count': 1.0}"
142,2022-05-19 00:11:37,2022-05-19 00:38:41,mp_wz_island,Duos,17873325042825069645,27,1,wz,144,70,...,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 65 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   utcStartSeconds                 144 non-null    datetime64[ns]
 1   utcEndSeconds                   144 non-null    datetime64[ns]
 2   map                             144 non-null    object        
 3   mode                            144 non-null    object        
 4   matchID                         144 non-null    object        
 5   duration                        144 non-null    object        
 6   version                         144 non-null    int64         
 7   gameType                        144 non-null    object        
 8   playerCount                     144 non-null    int64         
 9   teamCount                       144 non-null    int64         
 10  draw                            144 non-null    bool          
 11  privat

## Bonus : multiple calls

*COD API, does not like that very much though, particularly if concurrent, would like to know the exact rules ^_^*

In [ ]:
client = await callofduty.Login(sso=os.environ["SSO"])

In [ ]:
ids = await client.GetMatches('battle', 'amadevs#1689', Title.ModernWarfare, Mode.Warzone)
ids = [int(match['matchId']) for match in ids]
ids

In [ ]:
ids = ids[7:10]
ids

In [ ]:
#### Concurrently, using asyncio.gather()

In [ ]:
*Multiple concurrent calls with same session hit api limits pretty hard it seems*

In [ ]:
task_list = []
for match_id in ids:
    task_list.append(GetMatchStats(client, 'battle', Title.ModernWarfare, Mode.Warzone, matchId=match_id))
await asyncio.gather(*task_list)

In [ ]:
# note, see also https://github.com/florimondmanca/aiometer to manage concurrency strength or edit callofduty api

#### Preferred but slower way

In [ ]:
import time
detailed_matches = []

for id_ in ids:
    print(f"match id : {id_}")
    detailed_match = await client.GetMatchStats('battle', Title.ModernWarfare, Mode.Warzone, matchId=id_)
    time.sleep(5)
    detailed_match = res_to_df(detailed_match, CONF)
    detailed_match = format_df(detailed_match, CONF, LABELS)
    detailed_match = augment_df(detailed_match, LABELS)
    detailed_matches.append(detailed_match)

detailed_matches = pd.concat(detailed_matches)

In [ ]:
detailed_matches.head()

In [ ]:
with open("data/match_3.pkl", 'wb') as f:
    pickle.dump(detailed_matches, f)